In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

# File to Load
homicide_csv = Path("data/homicide_data.csv")

# Read homicide data file and store into Pandas DataFrame
homicide_data = pd.read_csv(homicide_csv, encoding='latin1')

In [2]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52179 entries, 0 to 52178
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            52179 non-null  object 
 1   reported_date  52179 non-null  int64  
 2   victim_last    52178 non-null  object 
 3   victim_first   52179 non-null  object 
 4   victim_race    52179 non-null  object 
 5   victim_age     52179 non-null  object 
 6   victim_sex     52179 non-null  object 
 7   city           52179 non-null  object 
 8   state          52179 non-null  object 
 9   lat            52119 non-null  float64
 10  lon            52119 non-null  float64
 11  disposition    52179 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 4.8+ MB


In [3]:
homicide_data['state'].unique()

array(['NM', 'GA', 'MD', 'LA', 'AL', 'MA', 'NY', 'NC', 'IL', 'OH', 'TX',
       'CO', 'MI', 'CA', 'IN', 'FL', 'MO', 'NV', 'KY', 'TN', 'wI', 'MN',
       'OK', 'NE', 'PA', 'AZ', 'VA', 'DC'], dtype=object)

In [4]:
#Wisconsin is WI no wI
homicide_data['state'] = homicide_data['state'].replace('wI', 'WI')

In [5]:
#Some states/cities did not disclose victim names
homicide_data =  homicide_data.drop(columns = ["victim_last", "victim_first"])

In [6]:
# Replace "unknown" values with NaN
homicide_data.replace('Unknown', pd.NA, inplace=True)

# Drop rows with NaN values in any column
homicide_data.dropna(axis=0, how='any', inplace=True)

# Now, 'homicide_data' contains rows where any column is not "Unknown"

In [7]:
homicide_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47478 entries, 0 to 52178
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            47478 non-null  object 
 1   reported_date  47478 non-null  int64  
 2   victim_race    47478 non-null  object 
 3   victim_age     47478 non-null  object 
 4   victim_sex     47478 non-null  object 
 5   city           47478 non-null  object 
 6   state          47478 non-null  object 
 7   lat            47478 non-null  float64
 8   lon            47478 non-null  float64
 9   disposition    47478 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 4.0+ MB


In [8]:
homicide_data.head()

uid  reported_date victim_race victim_age victim_sex         city  \
0  Alb-000001       20100504    Hispanic         78       Male  Albuquerque   
1  Alb-000002       20100216    Hispanic         17       Male  Albuquerque   
2  Alb-000003       20100601       White         15     Female  Albuquerque   
3  Alb-000004       20100101    Hispanic         32       Male  Albuquerque   
4  Alb-000005       20100102       White         72     Female  Albuquerque   

  state        lat         lon            disposition  
0    NM  35.095788 -106.538555  Closed without arrest  
1    NM  35.056810 -106.715321       Closed by arrest  
2    NM  35.086092 -106.695568  Closed without arrest  
3    NM  35.078493 -106.556094       Closed by arrest  
4    NM  35.130357 -106.580986  Closed without arrest

In [9]:
homicide_data.disposition = homicide_data.disposition.replace("Closed without arrest", "No Arrest")
homicide_data.disposition = homicide_data.disposition.replace("Open/No arrest", "No Arrest")
homicide_data.disposition = homicide_data.disposition.replace("Closed by arrest", "Arrest Made")

In [10]:
homicide_data.disposition.value_counts()

No Arrest      24258
Arrest Made    23220
Name: disposition, dtype: int64

In [11]:
homicide_data.columns

Index(['uid', 'reported_date', 'victim_race', 'victim_age', 'victim_sex',
       'city', 'state', 'lat', 'lon', 'disposition'],
      dtype='object')

In [12]:
homicide_data['reported_date'] = pd.to_datetime(homicide_data['reported_date'], format='%Y%m%d')

In [13]:
homicide_data['reported_year'] = homicide_data['reported_date'].dt.year
homicide_data['reported_month'] = homicide_data['reported_date'].dt.month
homicide_data['reported_weekday'] = homicide_data['reported_date'].dt.dayofweek

In [14]:
homicide_data['reported_weekday'].value_counts()

6    7850
5    7619
0    6853
4    6446
1    6331
2    6256
3    6123
Name: reported_weekday, dtype: int64

In [15]:
homicide_data.head()

uid reported_date victim_race victim_age victim_sex         city  \
0  Alb-000001    2010-05-04    Hispanic         78       Male  Albuquerque   
1  Alb-000002    2010-02-16    Hispanic         17       Male  Albuquerque   
2  Alb-000003    2010-06-01       White         15     Female  Albuquerque   
3  Alb-000004    2010-01-01    Hispanic         32       Male  Albuquerque   
4  Alb-000005    2010-01-02       White         72     Female  Albuquerque   

  state        lat         lon  disposition  reported_year  reported_month  \
0    NM  35.095788 -106.538555    No Arrest           2010               5   
1    NM  35.056810 -106.715321  Arrest Made           2010               2   
2    NM  35.086092 -106.695568    No Arrest           2010               6   
3    NM  35.078493 -106.556094  Arrest Made           2010               1   
4    NM  35.130357 -106.580986    No Arrest           2010               1   

   reported_weekday  
0                 1  
1                 1  
2                 1  
3                 4  
4                 5

In [16]:
# Convert 'victim_age' to numeric
homicide_data['victim_age'] = pd.to_numeric(homicide_data['victim_age'])

# Define the age bins and labels
age_bins = [0, 18, 30, 45, 64, float('inf')]
age_labels = ['0-17', '18-29', '30-44', '45-64', '65+']

# Bin the 'victim_age' column and create a new column 'age_range'
homicide_data['age_range'] = pd.cut(homicide_data['victim_age'], bins=age_bins, labels=age_labels, right=False)

In [17]:
homicide_data.age_range.value_counts()

18-29    21428
30-44    13461
45-64     7144
0-17      3916
65+       1529
Name: age_range, dtype: int64

In [18]:
# Define a function to map month values to seasons
def get_season(reported_month):
    if reported_month in [12,1,2]:
        return 'Winter'
    elif reported_month in [3, 4, 5]:
        return 'Spring'
    elif reported_month in [6, 7, 8]:
        return 'Summer'
    elif reported_month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Unknown'  # Handle invalid month values if any

# Apply the function to create a new "season" column
homicide_data['season'] = homicide_data['reported_month'].apply(get_season)

In [19]:
homicide_data.head()

uid reported_date victim_race  victim_age victim_sex         city  \
0  Alb-000001    2010-05-04    Hispanic          78       Male  Albuquerque   
1  Alb-000002    2010-02-16    Hispanic          17       Male  Albuquerque   
2  Alb-000003    2010-06-01       White          15     Female  Albuquerque   
3  Alb-000004    2010-01-01    Hispanic          32       Male  Albuquerque   
4  Alb-000005    2010-01-02       White          72     Female  Albuquerque   

  state        lat         lon  disposition  reported_year  reported_month  \
0    NM  35.095788 -106.538555    No Arrest           2010               5   
1    NM  35.056810 -106.715321  Arrest Made           2010               2   
2    NM  35.086092 -106.695568    No Arrest           2010               6   
3    NM  35.078493 -106.556094  Arrest Made           2010               1   
4    NM  35.130357 -106.580986    No Arrest           2010               1   

   reported_weekday age_range  season  
0                 1       65+  Spring  
1                 1      0-17  Winter  
2                 1      0-17  Summer  
3                 4     30-44  Winter  
4                 5       65+  Winter

In [20]:
# Create a new column 'LOCATION' by concatenating 'city' and 'state' with a comma separator
homicide_data['LOCATION'] = homicide_data['city'] + ', ' + homicide_data['state']

In [21]:
homicide_data['LOCATION'].unique()

array(['Albuquerque, NM', 'Atlanta, GA', 'Baltimore, MD',
       'Baton Rouge, LA', 'Birmingham, AL', 'Boston, MA', 'Buffalo, NY',
       'Charlotte, NC', 'Chicago, IL', 'Cincinnati, OH', 'Columbus, OH',
       'Denver, CO', 'Detroit, MI', 'Durham, NC', 'Fort Worth, TX',
       'Fresno, CA', 'Houston, TX', 'Indianapolis, IN',
       'Jacksonville, FL', 'Las Vegas, NV', 'Long Beach, CA',
       'Los Angeles, CA', 'Louisville, KY', 'Memphis, TN', 'Miami, FL',
       'Milwaukee, WI', 'Minneapolis, MN', 'Nashville, TN',
       'New Orleans, LA', 'New York, NY', 'Oakland, CA',
       'Oklahoma City, OK', 'Omaha, NE', 'Philadelphia, PA',
       'Pittsburgh, PA', 'Richmond, VA', 'San Antonio, TX',
       'Sacramento, CA', 'Savannah, GA', 'San Bernardino, CA',
       'San Diego, CA', 'San Francisco, CA', 'St. Louis, MO',
       'Stockton, CA', 'Tampa, FL', 'Tulsa, OK', 'Tulsa, AL',
       'Washington, DC'], dtype=object)

In [22]:
#There is no Tulsa, AL. The lon and lat indicate Oklahoma.
# Replace 'AL' with 'OK' for 'LOCATION' containing 'city' Tulsa.
homicide_data['LOCATION'] = homicide_data['LOCATION'].str.replace(r'Tulsa.*, AL$', 'Tulsa, OK', regex=True)

In [23]:
locations_to_keep = homicide_data['LOCATION'].unique()

In [24]:
# Specify the path for the new CSV file where you want to save the filtered data
output_csv = Path("data/filtered_homicide_data.csv")

# Save the filtered DataFrame to the new CSV file
homicide_data.to_csv(output_csv, index=False)

In [25]:
homicide_data.head()

uid reported_date victim_race  victim_age victim_sex         city  \
0  Alb-000001    2010-05-04    Hispanic          78       Male  Albuquerque   
1  Alb-000002    2010-02-16    Hispanic          17       Male  Albuquerque   
2  Alb-000003    2010-06-01       White          15     Female  Albuquerque   
3  Alb-000004    2010-01-01    Hispanic          32       Male  Albuquerque   
4  Alb-000005    2010-01-02       White          72     Female  Albuquerque   

  state        lat         lon  disposition  reported_year  reported_month  \
0    NM  35.095788 -106.538555    No Arrest           2010               5   
1    NM  35.056810 -106.715321  Arrest Made           2010               2   
2    NM  35.086092 -106.695568    No Arrest           2010               6   
3    NM  35.078493 -106.556094  Arrest Made           2010               1   
4    NM  35.130357 -106.580986    No Arrest           2010               1   

   reported_weekday age_range  season         LOCATION  
0                 1       65+  Spring  Albuquerque, NM  
1                 1      0-17  Winter  Albuquerque, NM  
2                 1      0-17  Summer  Albuquerque, NM  
3                 4     30-44  Winter  Albuquerque, NM  
4                 5       65+  Winter  Albuquerque, NM

***Population Data***

In [26]:
# Define the file paths using Path
Est_2010_csv = Path("data/SubEst2010.csv")
Est_2020_csv = Path("data/SubEst2020.csv")

# Load SubEst2010.csv into a DataFrame with the specified encoding
df_2010 = pd.read_csv(Est_2010_csv, encoding='latin1')

# Load SubEst2020.csv into a DataFrame with the specified encoding
df_2020 = pd.read_csv(Est_2020_csv, encoding='latin1')

In [27]:
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [28]:
df_2010.head()

SUMLEV  STATE  COUNTY  PLACE  COUSUB             NAME   STNAME  \
0      40      1       0      0       0          Alabama  Alabama   
1     162      1       0    124       0   Abbeville city  Alabama   
2     162      1       0    460       0  Adamsville city  Alabama   
3     162      1       0    484       0     Addison town  Alabama   
4     162      1       0    676       0       Akron town  Alabama   

   ESTIMATESBASE2000  POPESTIMATE2000  POPESTIMATE2001  POPESTIMATE2002  \
0            4447207          4452173          4467634          4480089   
1               2989             2985             2941             2909   
2               5033             5021             4960             4894   
3                698              701              701              708   
4                488              485              473              454   

   POPESTIMATE2003  POPESTIMATE2004  POPESTIMATE2005  POPESTIMATE2006  \
0          4503491          4530729          4569805          4628981   
1             2882             2857             2820             2807   
2             4841             4784             4728             4687   
3              714              722              729              741   
4              442              426              416              404   

   POPESTIMATE2007  POPESTIMATE2008  POPESTIMATE2009  CENSUS2010POP  \
0          4672840          4718206          4757938        4779736   
1             2784             2742             2714           2688   
2             4633             4594             4558           4522   
3              750              752              759            758   
4              395              384              369            356   

   POPESTIMATE2010  
0          4785298  
1             2689  
2             4523  
3              755  
4              355

In [29]:
df_2010.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'NAME', 'STNAME',
       'ESTIMATESBASE2000', 'POPESTIMATE2000', 'POPESTIMATE2001',
       'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004',
       'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007',
       'POPESTIMATE2008', 'POPESTIMATE2009', 'CENSUS2010POP',
       'POPESTIMATE2010'],
      dtype='object')

In [30]:
df_2010['STNAME'] = df_2010['STNAME'].replace(state_abbreviations)

In [31]:
df_2010.head()

SUMLEV  STATE  COUNTY  PLACE  COUSUB             NAME STNAME  \
0      40      1       0      0       0          Alabama     AL   
1     162      1       0    124       0   Abbeville city     AL   
2     162      1       0    460       0  Adamsville city     AL   
3     162      1       0    484       0     Addison town     AL   
4     162      1       0    676       0       Akron town     AL   

   ESTIMATESBASE2000  POPESTIMATE2000  POPESTIMATE2001  POPESTIMATE2002  \
0            4447207          4452173          4467634          4480089   
1               2989             2985             2941             2909   
2               5033             5021             4960             4894   
3                698              701              701              708   
4                488              485              473              454   

   POPESTIMATE2003  POPESTIMATE2004  POPESTIMATE2005  POPESTIMATE2006  \
0          4503491          4530729          4569805          4628981   
1             2882             2857             2820             2807   
2             4841             4784             4728             4687   
3              714              722              729              741   
4              442              426              416              404   

   POPESTIMATE2007  POPESTIMATE2008  POPESTIMATE2009  CENSUS2010POP  \
0          4672840          4718206          4757938        4779736   
1             2784             2742             2714           2688   
2             4633             4594             4558           4522   
3              750              752              759            758   
4              395              384              369            356   

   POPESTIMATE2010  
0          4785298  
1             2689  
2             4523  
3              755  
4              355

In [32]:
# Remove ' city' and ' town' from the 'NAME' column
df_2010['NAME'] = df_2010['NAME'].str.replace(r' city.*$', '', regex=True)
df_2010['NAME'] = df_2010['NAME'].str.replace(r' town.*$', '', regex=True)
df_2010['NAME'] = df_2010['NAME'].str.replace(r' village.*$', '', regex=True)
df_2010['NAME'] = df_2010['NAME'].str.replace(r'/.*$', '', regex=True)
df_2010['NAME'] = df_2010['NAME'].str.replace(r'-.*$', '', regex=True)

In [33]:
# Create a new column 'LOCATION' by concatenating 'NAME' and 'STNAME' with a comma separator
df_2010['LOCATION'] = df_2010['NAME'] + ', ' + df_2010['STNAME']

In [34]:
# Select and retain only the desired columns
columns_to_keep_2010 = ['NAME', 'STNAME', 'LOCATION', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009']
df_2010 = df_2010[columns_to_keep_2010]

In [35]:
df_2010.head()

NAME STNAME        LOCATION  POPESTIMATE2007  POPESTIMATE2008  \
0     Alabama     AL     Alabama, AL          4672840          4718206   
1   Abbeville     AL   Abbeville, AL             2784             2742   
2  Adamsville     AL  Adamsville, AL             4633             4594   
3     Addison     AL     Addison, AL              750              752   
4       Akron     AL       Akron, AL              395              384   

   POPESTIMATE2009  
0          4757938  
1             2714  
2             4558  
3              759  
4              369

In [36]:
df_2020.head()

SUMLEV  STATE  COUNTY  PLACE  COUSUB  CONCIT  PRIMGEO_FLAG FUNCSTAT  \
0      40      1       0      0       0       0             0        A   
1     162      1       0    124       0       0             0        A   
2     162      1       0    460       0       0             0        A   
3     162      1       0    484       0       0             0        A   
4     162      1       0    676       0       0             0        A   

              NAME   STNAME  ... POPESTIMATE2012  POPESTIMATE2013  \
0          Alabama  Alabama  ...         4816632          4831586   
1   Abbeville city  Alabama  ...            2645             2629   
2  Adamsville city  Alabama  ...            4453             4430   
3     Addison town  Alabama  ...             745              744   
4       Akron town  Alabama  ...             347              344   

   POPESTIMATE2014  POPESTIMATE2015  POPESTIMATE2016  POPESTIMATE2017  \
0          4843737          4854803          4866824          4877989   
1             2610             2602             2587             2578   
2             4399             4371             4335             4304   
3              742              734              734              728   
4              338              338              335              332   

   POPESTIMATE2018  POPESTIMATE2019  POPESTIMATE042020  POPESTIMATE2020  
0          4891628          4907965            4920706          4921532  
1             2565             2555               2555             2553  
2             4285             4254               4224             4211  
3              725              723                719              717  
4              332              328                328              327  

[5 rows x 24 columns]

In [37]:
df_2020.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'CONCIT',
       'PRIMGEO_FLAG', 'FUNCSTAT', 'NAME', 'STNAME', 'CENSUS2010POP',
       'ESTIMATESBASE2010', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019', 'POPESTIMATE042020',
       'POPESTIMATE2020'],
      dtype='object')

In [38]:
df_2020['STNAME'] = df_2020['STNAME'].replace(state_abbreviations)

In [39]:
# Remove ' city' and ' town' from the 'NAME' column
df_2020['NAME'] = df_2020['NAME'].str.replace(r' city.*$', '', regex=True)
df_2020['NAME'] = df_2020['NAME'].str.replace(r' town.*$', '', regex=True)
df_2020['NAME'] = df_2020['NAME'].str.replace(r' village.*$', '', regex=True)
df_2020['NAME'] = df_2020['NAME'].str.replace(r'/.*$', '', regex=True)
df_2020['NAME'] = df_2020['NAME'].str.replace(r'-.*$', '', regex=True)

In [40]:
# Create a new column 'LOCATION' by concatenating 'NAME' and 'STNAME' with a comma separator
df_2020['LOCATION'] = df_2020['NAME'] + ', ' + df_2020['STNAME']

In [41]:
# Select and retain only the desired columns
columns_to_keep_2020 = ['NAME', 'STNAME', 'LOCATION', 'CENSUS2010POP', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017']
df_2020 = df_2020[columns_to_keep_2020]

In [42]:
df_2020.head()

NAME STNAME        LOCATION CENSUS2010POP  POPESTIMATE2011  \
0     Alabama     AL     Alabama, AL       4779736          4799642   
1   Abbeville     AL   Abbeville, AL          2688             2694   
2  Adamsville     AL  Adamsville, AL          4522             4474   
3     Addison     AL     Addison, AL           758              750   
4       Akron     AL       Akron, AL           356              347   

   POPESTIMATE2012  POPESTIMATE2013  POPESTIMATE2014  POPESTIMATE2015  \
0          4816632          4831586          4843737          4854803   
1             2645             2629             2610             2602   
2             4453             4430             4399             4371   
3              745              744              742              734   
4              347              344              338              338   

   POPESTIMATE2016  POPESTIMATE2017  
0          4866824          4877989  
1             2587             2578  
2             4335             4304  
3              734              728  
4              335              332

In [43]:
merged_df = df_2010.merge(df_2020, on=['NAME', 'STNAME', 'LOCATION'], how='inner')

In [44]:
merged_df.columns

Index(['NAME', 'STNAME', 'LOCATION', 'POPESTIMATE2007', 'POPESTIMATE2008',
       'POPESTIMATE2009', 'CENSUS2010POP', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017'],
      dtype='object')

In [45]:
# Create a mapping dictionary for column renaming
column_name_mapping = {
    'POPESTIMATE2007': '2007',
    'POPESTIMATE2008': '2008',
    'POPESTIMATE2009': '2009',
    'CENSUS2010POP': '2010',
    'POPESTIMATE2011': '2011',
    'POPESTIMATE2012': '2012',
    'POPESTIMATE2013': '2013',
    'POPESTIMATE2014': '2014',
    'POPESTIMATE2015': '2015',
    'POPESTIMATE2016': '2016',
    'POPESTIMATE2017': '2017'
}

In [46]:
# Rename the columns based on the mapping dictionary
merged_df = merged_df.rename(columns=column_name_mapping)

In [47]:
merged_df.head()

NAME STNAME       LOCATION     2007     2008     2009     2010  \
0    Alabama     AL    Alabama, AL  4672840  4718206  4757938  4779736   
1  Abbeville     AL  Abbeville, AL     2784     2742     2714     2688   
2  Abbeville     AL  Abbeville, AL     2784     2742     2714     2688   
3  Abbeville     AL  Abbeville, AL     2784     2742     2714     2688   
4  Abbeville     AL  Abbeville, AL     2784     2742     2714     2688   

      2011     2012     2013     2014     2015     2016     2017  
0  4799642  4816632  4831586  4843737  4854803  4866824  4877989  
1     2694     2645     2629     2610     2602     2587     2578  
2     2694     2645     2629     2610     2602     2587     2578  
3     2694     2645     2629     2610     2602     2587     2578  
4     2694     2645     2629     2610     2602     2587     2578

In [48]:
# Remove duplicate rows based on 'LOCATION' column
merged_df = merged_df.drop_duplicates(subset=['LOCATION'])

In [49]:
merged_df.head()

NAME STNAME        LOCATION     2007     2008     2009     2010  \
0      Alabama     AL     Alabama, AL  4672840  4718206  4757938  4779736   
1    Abbeville     AL   Abbeville, AL     2784     2742     2714     2688   
5   Adamsville     AL  Adamsville, AL     4633     4594     4558     4522   
9      Addison     AL     Addison, AL      750      752      759      758   
13       Akron     AL       Akron, AL      395      384      369      356   

       2011     2012     2013     2014     2015     2016     2017  
0   4799642  4816632  4831586  4843737  4854803  4866824  4877989  
1      2694     2645     2629     2610     2602     2587     2578  
5      4474     4453     4430     4399     4371     4335     4304  
9       750      745      744      742      734      734      728  
13      347      347      344      338      338      335      332

In [50]:
# Find the row where 'LOCATION' is equal to 'Your_Location'
matching_rows = merged_df[merged_df['LOCATION'].str.contains('Tulsa', case=False, na=False)]

# Display the specific location
print(matching_rows)

                           NAME STNAME                     LOCATION    2007  \
204972                    Tulsa     OK                    Tulsa, OK  385779   
205371             Tulsa County     OK             Tulsa County, OK  582094   
205372  Balance of Tulsa County     OK  Balance of Tulsa County, OK   33388   

          2008    2009    2010    2011    2012    2013    2014    2015  \
204972  387130  390339  391906  392937  394616  398419  399945  403576   
205371  588510  597748  603403  609323  615376  623978  631241  640851   
205372   33845   34450   34828   34197   34378   34809   35162   35600   

          2016    2017  
204972  404086  402177  
205371  646328  646874  
205372   35878   35954  


In [51]:
filtered_pop_df = merged_df[merged_df['LOCATION'].isin(locations_to_keep)]
filtered_pop_df

NAME STNAME            LOCATION      2007      2008  \
180         Birmingham     AL      Birmingham, AL    218880    216505   
6143            Fresno     CA          Fresno, CA    477659    484443   
6475        Long Beach     CA      Long Beach, CA    460328    460643   
6495       Los Angeles     CA     Los Angeles, CA   3751872   3763566   
6699           Oakland     CA         Oakland, CA    383500    386589   
6971        Sacramento     CA      Sacramento, CA    455316    459241   
6987    San Bernardino     CA  San Bernardino, CA    209084    208921   
7011         San Diego     CA       San Diego, CA   1266198   1282692   
7023     San Francisco     CA   San Francisco, CA    778673    791152   
7223          Stockton     CA        Stockton, CA    285313    286507   
7880            Denver     CO          Denver, CO    564395    575721   
9745        Washington     DC      Washington, DC    574404    580236   
10432     Jacksonville     FL    Jacksonville, FL    807669    812430   
10726            Miami     FL           Miami, FL    386814    389996   
11239            Tampa     FL           Tampa, FL    332024    332173   
11684          Atlanta     GA         Atlanta, GA    410086    414233   
13494         Savannah     GA        Savannah, GA    133452    133651   
19497          Chicago     IL         Chicago, IL   2703991   2697359   
45916     Indianapolis     IN    Indianapolis, IN    802751    808013   
89927       Louisville     KY      Louisville, KY    581388    587675   
91159      Baton Rouge     LA     Baton Rouge, LA    228814    228177   
91935      New Orleans     LA     New Orleans, LA    268751    301842   
93377        Baltimore     MD       Baltimore, MD    620306    620184   
94143           Boston     MA          Boston, MA    593136    600685   
97474          Detroit     MI         Detroit, MI    784998    756383   
118342     Minneapolis     MN     Minneapolis, MN    377575    379185   
133655       St. Louis     MO       St. Louis, MO    317959    317955   
139405           Omaha     NE           Omaha, NE    405904    406320   
141659       Las Vegas     NV       Las Vegas, NV    578302    584400   
147755     Albuquerque     NM     Albuquerque, NM    524090    531451   
148242        New York     NY        New York, NY  19132335  19212436   
149267         Buffalo     NY         Buffalo, NY    268358    265732   
158274       Charlotte     NC       Charlotte, NC    691568    709993   
158456          Durham     NC          Durham, NC    215097    220897   
171812      Cincinnati     OH      Cincinnati, OH    301824    299145   
172234        Columbus     OH        Columbus, OH    760224    770530   
204357   Oklahoma City     OK   Oklahoma City, OK    554280    561926   
204972           Tulsa     OK           Tulsa, OK    385779    387130   
218317    Philadelphia     PA    Philadelphia, PA   1493309   1499731   
218417      Pittsburgh     PA      Pittsburgh, PA    308999    307673   
238769         Memphis     TN         Memphis, TN    659816    653107   
238887       Nashville     TN       Nashville, TN    580162    587567   
241442      Fort Worth     TX      Fort Worth, TX    685762    706526   
242058         Houston     TX         Houston, TX   2044612   2060477   
244287     San Antonio     TX     San Antonio, TX   1270187   1291232   
248754        Richmond     VA        Richmond, VA    200201    202254   
260113       Milwaukee     WI       Milwaukee, WI    586445    587858   

            2009      2010      2011      2012      2013      2014      2015  \
180       214394    212237    211554    210775    211270    211179    211811   
6143      490262    494665    501623    505554    508993    514052    517408   
6475      461782    462257    464274    466526    468017    468785    469408   
6495     3781938   3792621   3818812   3847857   3877721   3904102   3933644   
6699      389613    390724    395416    400720    406609    412677    418539   
6971      463458    466488    470

In [52]:
# Specify the path for the new CSV file where you want to save the filtered data
output_csv = Path("data/filtered_pop_data.csv")

# Save the filtered DataFrame to the new CSV file
filtered_pop_df.to_csv(output_csv, index=False, header=True)

***Create a new table for specific date***

In [53]:
filtered_pop_df.columns

Index(['NAME', 'STNAME', 'LOCATION', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017'],
      dtype='object')

In [54]:
id_vars = ['NAME', 'STNAME', 'LOCATION']
value_vars = ['2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017']

In [55]:
df_melt = pd.melt(filtered_pop_df,id_vars=id_vars,value_vars=value_vars,var_name="YEAR", value_name="POPULATION")
df_melt['YEAR'] = df_melt.YEAR.astype(int)
df_melt['POPULATION'] = df_melt.POPULATION.astype(int)

In [56]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NAME        517 non-null    object
 1   STNAME      517 non-null    object
 2   LOCATION    517 non-null    object
 3   YEAR        517 non-null    int32 
 4   POPULATION  517 non-null    int32 
dtypes: int32(2), object(3)
memory usage: 16.3+ KB


In [57]:
df_final = pd.merge(homicide_data, df_melt, left_on = ['LOCATION','reported_year'],
                    right_on = ['LOCATION', 'YEAR'], how = 'left')

In [58]:
df_final.head()

uid reported_date victim_race  victim_age victim_sex         city  \
0  Alb-000001    2010-05-04    Hispanic          78       Male  Albuquerque   
1  Alb-000002    2010-02-16    Hispanic          17       Male  Albuquerque   
2  Alb-000003    2010-06-01       White          15     Female  Albuquerque   
3  Alb-000004    2010-01-01    Hispanic          32       Male  Albuquerque   
4  Alb-000005    2010-01-02       White          72     Female  Albuquerque   

  state        lat         lon  disposition  reported_year  reported_month  \
0    NM  35.095788 -106.538555    No Arrest           2010               5   
1    NM  35.056810 -106.715321  Arrest Made           2010               2   
2    NM  35.086092 -106.695568    No Arrest           2010               6   
3    NM  35.078493 -106.556094  Arrest Made           2010               1   
4    NM  35.130357 -106.580986    No Arrest           2010               1   

   reported_weekday age_range  season         LOCATION         NAME STNAME  \
0                 1       65+  Spring  Albuquerque, NM  Albuquerque     NM   
1                 1      0-17  Winter  Albuquerque, NM  Albuquerque     NM   
2                 1      0-17  Summer  Albuquerque, NM  Albuquerque     NM   
3                 4     30-44  Winter  Albuquerque, NM  Albuquerque     NM   
4                 5       65+  Winter  Albuquerque, NM  Albuquerque     NM   

   YEAR  POPULATION  
0  2010      545852  
1  2010      545852  
2  2010      545852  
3  2010      545852  
4  2010      545852

In [59]:
df_per = df_final.groupby(["LOCATION","reported_year"]).agg({"uid":"size","POPULATION": 'max'}).reset_index()
df_per["homicides_per_100000"] = df_per["uid"]/df_per["POPULATION"]*100000

df_per

LOCATION  reported_year  uid  POPULATION  homicides_per_100000
0    Albuquerque, NM           2010   43      545852              7.877593
1    Albuquerque, NM           2011   37      552105              6.701624
2    Albuquerque, NM           2012   44      555074              7.926871
3    Albuquerque, NM           2013   32      557547              5.739426
4    Albuquerque, NM           2014   27      557566              4.842476
..               ...            ...  ...         ...                   ...
464   Washington, DC           2012   88      635737             13.842202
465   Washington, DC           2013  104      651559             15.961716
466   Washington, DC           2014  105      663603             15.822713
467   Washington, DC           2015  162      677014             23.928604
468   Washington, DC           2016  135      687576             19.634193

[469 rows x 5 columns]

In [60]:
# Specify the path for the new CSV file where you want to save the filtered data
output_csv = Path("data/filtered_per_100000.csv")

# Save the filtered DataFrame to the new CSV file
df_per.to_csv(output_csv, index=False, header=True)